In [21]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")


In [22]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [ ]:
import os
from langchain_openai import ChatOpenAI

# Set API key
os.environ["OPENAI_API_KEY"] = "your-api-key"

llm = ChatOpenAI(model="gpt-4o-mini")

In [32]:
# Function to generate slide content using AI
def generate_slide_content(topic):
    prompt = f"Generate a structured 5-slide PowerPoint outline for the topic: {topic}."
    response = llm.invoke(prompt).content  # Ensure we get text output
    return response.split("\n")  # Split into slides

In [50]:
import tempfile


In [60]:
def generate_pptx(topic):
    prs = Presentation()
    slides = generate_slide_content(topic)

    for slide_text in slides[:5]:  
        if ":" in slide_text:
            title, content = slide_text.split(":", 1)
        else:
            title, content = slide_text, ""

        slide_layout = prs.slide_layouts[1]  # Title & Content Layout
        slide = prs.slides.add_slide(slide_layout)
        slide.shapes.title.text = title.strip()
        slide.placeholders[1].text = content.strip()

    # Save to a known directory (e.g., current working directory)
    file_path = os.path.join(os.getcwd(), f"{topic.replace(' ', '_')}.pptx")
    prs.save(file_path)

    print(f"PPTX file saved at: {file_path}")
    return file_path


In [61]:
# AI function to generate PPTX
def chatbot(state: State):
    topic = state["messages"][-1]  # Get latest user input
    
    # Check if input is a string
    if hasattr(topic, "content"):  
        topic = topic.content  # ✅ Extract string if needed

    pptx_path = generate_pptx(topic)
    return {"messages": [f"Here is your PPT on {topic}: {pptx_path}"]}


In [62]:
# Define LangGraph workflow
graph_builder = StateGraph(State)

In [63]:
graph_builder.add_node("chatbot", chatbot)

In [64]:
graph_builder.set_entry_point("chatbot")

In [65]:
# Compile the workflow
ppt_workflow = graph_builder.compile()

In [66]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [68]:
# Example Test: User requests a PPT
test_state = {"messages": ["Artificial Intelligence"]}
output = ppt_workflow.invoke(test_state)

PPTX file saved at: e:\aXtrWorks\LangGraph\Artificial_Intelligence.pptx


In [69]:
# Display response
print(output["messages"][0])

content='Artificial Intelligence' additional_kwargs={} response_metadata={} id='8da070a2-b199-434b-99df-cc3253a07a1c'
